In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import zipfile
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set()

sns.set_context("talk")
import re

# Ensure that Pandas shows at least 100 characters in columns
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

from pathlib import Path

# SQL
import sqlalchemy

In [11]:
from importlib import reload
import utils as utils
reload(utils)

<module 'utils' from '/scratch/public/btruong/utils.py'>

In [12]:
# Read LA Unsubsidized
la = pd.read_csv('los_angeles_u'+'_v2.csv', sep='|')

/usr/local/linux/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,14,16,17,19,22,30,40,43,44,45,48,51,53,56,57,60,63,73,74,79,92,94,99,100,103,120,121,123,124,131) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
display(la.head())
print(la.shape)

,RowID,ImportParcelID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,DupAPN,ParcelSequenceNumber,ParcelNumberTypeStndCode,RecordSourceStndCode,RecordTypeStndCode,ConfidentialRecordFlag,PropertyAddressSourceStndCode,PropertyHouseNumber,PropertyHouseNumberExt,PropertyStreetPreDirectional,PropertyStreetName,PropertyStreetSuffix,PropertyStreetPostDirectional,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyZip,OriginalPropertyFullStreetAddress,OriginalPropertyAddressLastline,PropertyBuildingNumber,PropertyZoningDescription,PropertyZoningSourceCode,CensusTract,TaxIDNumber,TaxAmount,TaxYear,TaxDelinquencyFlag,TaxDelinquencyAmount,TaxDelinquencyYear,TaxRateCodeArea,LegalLot,LegalLotStndCode,LegalOtherLot,LegalBlock,LegalSubdivisionCode,LegalSubdivisionName,LegalCondoProjectPUDDevName,LegalBuildingNumber,LegalUnit,LegalSection,LegalPhase,LegalTract,LegalDistrict,LegalMunicipality,LegalCity,LegalTownship,LegalSTRSection,LegalSTRTownship,LegalSTRRange,LegalSTRMeridian,LegalSecTwnRngMer,LegalRecordersMapReference,LegalDescription,LegalNeighborhoodSourceCode,NoOfBuildings,LotSizeAcres,LotSizeSquareFeet,LotSizeFrontageFeet,LotSizeDepthFeet,LotSizeIRR,LotSiteTopographyStndCode,LoadID,PropertyAddressMatchcode,PropertyAddressUnitDesignator,PropertyAddressUnitNumber,PropertyAddressCarrierRoute,PropertyAddressGeoCodeMatchCode,PropertyAddressLatitude,PropertyAddressLongitude,PropertyAddressCensusTractAndBlock,PropertyAddressConfidenceScore,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,RowID2,NoOfUnits,OccupancyStatusStndCode,PropertyCountyLandUseDescription,PropertyCountyLandUseCode,PropertyLandUseStndCode,PropertyStateLandUseDescription,PropertyStateLandUseCode,BuildingOrImprovementNumber,BuildingClassStndCode,BuildingQualityStndCode,BuildingQualityStndCodeOriginal,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,BathSourceStndCode,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FirePlaceTypeStndCode,FireplaceNumber,WaterStndCode,SewerStndCode,MortgageLenderName,TimeshareStndCode,Comments,StoryTypeStndCode,merge_row,Subsidized
0,890C553A-34D5-E511-80C1-3863BB43AC67,11178348,6037,CA,LOS ANGELES,NaN,82002,10,BKF,2057-022-003,NaN,1,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WVR115000*,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,9-036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR=43744 THAT POR IN TRA 9036 OF LOT 1 N,NaN,NaN,NaN,41600.0,NaN,NaN,NaN,NaN,367352071.0,N,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,324640.0,-1.0,890C553A-34D5-E511-80C1-3863BB43AC67,NaN,NaN,VACANT RESIDENTIAL LAND,010V,VL101,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,N
1,0317553A-34D5-E511-80C1-3863BB43AC67,11181372,6037,CA,LOS ANGELES,NaN,82002,10,BKF,2061-007-056,NaN,1,NaN,NaN,NaN,,NaN,28632,NaN,NaN,ROADSIDE,DR,NaN,28632 ROADSIDE DR,AGOURA HILLS,CA,91301.0,NaN,NaN,NaN,AH CRS-FC-OA*,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,11-464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L S 15-8-9 FOR DESC SEE ASSESSOR'S MAPS POR OF LOT 24,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,367350808.0,Y,NaN,NaN,C005,Y,34.1436,-118.749,060378003.032016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,324640.0,-1.0,0317553A-34D5-E511-80C1-3863BB43AC67,1.0,NaN,SINGLE FAMILY RESIDENTIAL,0100,RR101,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1951.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

(2174449, 135)


In [14]:
# Clean and filter for Santa Monica JCE only
sm_jc = utils.clean_city(la, 'SANTA MONICA')
print('Current number:', sm_jc.NoOfUnits.sum())

16.67% current number: 50635.0
33.33% current number: 50635.0
50.0% current number: 50531.0
66.67% current number: 51020.5
83.33% current number: 51020.5
100.0% current number: 51037.5
Current number: 51037.5


In [15]:
# Filter for not subsidized
sm_jc = sm_jc[sm_jc['Subsidized'] == 'N']

In [16]:
# Check structure 
# (number of records, number of columns)
sm_jc.shape

(21711, 135)

In [17]:
"""
-Owner-occupied dwellings with no more than three (3) units.
"""

sm_jc = sm_jc[~(sm_jc['OccupancyStatusStndCode'].isin(['O', 'I']) & ((sm_jc.NoOfUnits  <= 3) | (sm_jc.PropertyLandUseStndCode.isin(['RI101', 'RI102']))))]
print('Current number:', sm_jc.NoOfUnits.sum())

Current number: 36552.5


In [18]:
# -Rental units and dwellings constructed after the adoption of this Article; this exemption does not apply to units created as a result of conversion as opposed to new construction.
sm_jc = sm_jc[sm_jc.YearBuilt < 1980]
print('Current number:', sm_jc.NoOfUnits.sum())

Current number: 30668.5


# Rent Control

In [19]:
# Clean and filter for Santa Monica RC only
sm_rc = utils.clean_city(la, 'SANTA MONICA')
print('Current number:', sm_rc.NoOfUnits.sum())

16.67% current number: 50635.0
33.33% current number: 50635.0
50.0% current number: 50531.0
66.67% current number: 51020.5
83.33% current number: 51020.5
100.0% current number: 51037.5
Current number: 51037.5


In [21]:
# Filter for not subsidized
sm_rc = sm_rc[sm_rc['Subsidized'] == 'N']
print('Current number:', sm_rc.NoOfUnits.sum())

Current number: 51037.5


In [22]:
"""
Eligible units for RC: any unit built before 1979, residential;
"""
sm_rc = sm_rc[sm_rc.YearBuilt < 1979]
print('Current number:', sm_rc.NoOfUnits.sum())

Current number: 41237.5


In [29]:
"""
Non-eligible units for RC: Units built after 1979, single family homes, 
"""
sm_rc = sm_rc[~sm_rc.PropertyLandUseStndCode.isin(['RR101', 'RR999', 'RR106', 'RR107', 'RI113'])]
print('Current number:', sm_rc.NoOfUnits.sum())

Current number: 28422.5


In [30]:
print('Last JC Current number:', sm_jc.NoOfUnits.sum())
# includes full and partially coverage
print('Last RC Current number:', sm_rc.NoOfUnits.sum())

Last JC Current number: 30668.5
Last RC Current number: 28422.5


In [ ]:
la[la['Subsidized'] == 'Y'].shape

In [ ]:
"

Santa Monica:
Current JCE total unit number: 30668.5
Current RC total unit number: 28422.5

Berkeley:
Current JCE total unit number: 64742.0
Current RC total unit number: 23803.0

"